In [10]:
# Import necessary libraries
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
from gensim.models import Word2Vec
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
import ast
import re
import time
import xgboost as xgb

# Load datasets
train_df = pd.read_csv('../Data/train.csv')
test_df = pd.read_csv('../Data/test.csv')

# Preprocess text
def clean_text(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)  # Remove digits
    text = text.translate(str.maketrans('', '', string.punctuation))
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stopwords.words('english')]
    return tokens

train_df['review'] = train_df['review'].apply(clean_text)
test_df['review'] = test_df['review'].apply(clean_text)

In [11]:
# Combine train and test datasets for Word2Vec training
combined_reviews = pd.concat([train_df['review'], test_df['review']], axis=0)

# Train Word2Vec model on combined data
word2vec_model = Word2Vec(combined_reviews.tolist(), vector_size=100, window=5, min_count=2, workers=4)

# Save Word2Vec model
word2vec_model.save("custom_word2vec.model")

In [12]:
word2vec_model = Word2Vec.load("custom_word2vec.model")

# Function to create averaged word vectors
def feature_vector(words, model):
    featureVec = np.zeros((model.vector_size,), dtype="float32")
    nwords = 0
    index2word_set = set(model.wv.index_to_key)
    for word in words:
        if word in index2word_set:
            nwords += 1
            featureVec = np.add(featureVec, model.wv[word])
    if nwords > 0:
        featureVec = np.divide(featureVec, nwords)
    return featureVec

def get_avg_feature_vectors(reviews, model):
    reviewFeatureVecs = np.zeros((len(reviews), model.vector_size), dtype="float32")
    for i, review in enumerate(reviews):
        reviewFeatureVecs[i] = feature_vector(review, model)
    return reviewFeatureVecs

# Generate feature vectors for both train and test data
trainDataVecs = get_avg_feature_vectors(train_df['review'], word2vec_model)
testDataVecs = get_avg_feature_vectors(test_df['review'], word2vec_model)

# Save feature vectors
np.save('trainDataVecs.npy', trainDataVecs)
np.save('testDataVecs.npy', testDataVecs)

In [13]:
# Encode sentiment labels
le = LabelEncoder()
y_train = le.fit_transform(train_df['sentiment'])
y_test = le.transform(test_df['sentiment'])

# Save labels
np.save('y_train.npy', y_train)
np.save('y_test.npy', y_test)

# Function to train and evaluate a model
def train_evaluate_model(model, X_train, y_train, X_test, y_test, model_name='Model'):
    start = time.time()
    model.fit(X_train, y_train)
    end = time.time()
    predictions = model.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    print(f"{model_name} Accuracy: {accuracy:.4f}")
    print(f"{model_name} Time: {end-start:.2f} seconds")

# Load feature vectors and labels for training
X_train = np.load('trainDataVecs.npy')
X_test = np.load('testDataVecs.npy')
y_train = np.load('y_train.npy')
y_test = np.load('y_test.npy')

In [14]:
# Train and evaluate Random Forest
rf = RandomForestClassifier(n_estimators=100)
train_evaluate_model(rf, X_train, y_train, X_test, y_test, 'Random Forest')

Random Forest Accuracy: 0.8310
Random Forest Time: 30.65 seconds


In [15]:
# Train and evaluate Naive Bayes
gnb = GaussianNB()
train_evaluate_model(gnb, X_train, y_train, X_test, y_test, 'Naive Bayes')

Naive Bayes Accuracy: 0.7663
Naive Bayes Time: 0.01 seconds


In [16]:
# Train and evaluate k-NN
knn = KNeighborsClassifier(n_neighbors=5)
train_evaluate_model(knn, X_train, y_train, X_test, y_test, 'k-NN')

k-NN Accuracy: 0.7994
k-NN Time: 0.00 seconds


In [17]:
# Create the Voting Classifier ensemble
voting_clf = VotingClassifier(estimators=[
    ('rf', rf), ('gnb', gnb), ('knn', knn)],
    voting='hard')

# Train and evaluate the Voting Classifier
train_evaluate_model(voting_clf, X_train, y_train, X_test, y_test, 'Voting Classifier')

Voting Classifier Accuracy: 0.8195
Voting Classifier Time: 29.23 seconds


In [18]:
# %%
# Train and evaluate XGBoost
xgb_model = xgb.XGBClassifier(objective='binary:logistic', eval_metric='logloss', use_label_encoder=False)
train_evaluate_model(xgb_model, X_train, y_train, X_test, y_test, 'XGBoost')

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


XGBoost Accuracy: 0.8500
XGBoost Time: 19.01 seconds
